In [9]:
# STEP 1: Installing required libraries 
!pip install flask pyngrok twilio --quiet

In [10]:
# STEP 2: Setting up SQLite database
import sqlite3
from datetime import datetime

conn = sqlite3.connect("expenses.db", check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS expenses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user TEXT,
    amount INTEGER,
    category TEXT,
    vendor TEXT,
    description TEXT,
    date TEXT
)
''')
conn.commit()
print(" SQLite database and 'expenses' table ready.")

 SQLite database and 'expenses' table ready.


In [11]:
# STEP 3: Defining a function to save expenses
def save_expense(user, amount, category, vendor, description, date=None):
    if not date:
        date = datetime.now().strftime("%Y-%m-%d")
    cursor.execute('''
        INSERT INTO expenses (user, amount, category, vendor, description, date)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (user, amount, category, vendor, description, date))
    conn.commit()


In [12]:
import re

In [13]:
def detect_category(msg):
    msg = msg.lower()

    category_keywords = {
        "food": ["tea", "coffee", "pizza", "biryani", "snacks", "breakfast", "lunch", "dinner", "swiggy", "zomato", "ccd", "starbucks","pizza"],
        "essentials": ["groceries", "milk", "vegetables", "zepto", "blinkit", "amazon pantry"],
        "medicines": ["pharmacy", "tablet", "medicine", "1mg", "medlife", "prescription", "chemist"],
        "sports": ["gym", "shoes", "football", "cricket", "bat", "sports", "yoga", "exercise","jersey"]
    }

    for category, keywords in category_keywords.items():
        if any(word in msg for word in keywords):
            return category

    return "other"  # fallback


In [14]:
# STEP 4: Creating Flask app and webhook
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
import re

app = Flask(__name__)

@app.route("/webhook", methods=["POST"])
def webhook():
    import re
    from datetime import datetime

    incoming_msg = request.form.get('Body') or ""
    sender = request.form.get('From') or ""
    msg_lower = incoming_msg.lower()

    print(" Incoming:", incoming_msg)

    response = MessagingResponse()
    msg = response.message()

    # STEP 1: Handle expense messages
    if any(word in msg_lower for word in ["spent", "paid", "bought"]):
        amount_match = re.findall(r'(?:₹|rs)?\s?(\d+)', msg_lower)
        if amount_match:
            amount = int(amount_match[0])
            category = detect_category(incoming_msg)  # Auto-categorize
            vendor = "unknown"
            description = incoming_msg
            date = datetime.now().strftime("%Y-%m-%d")

            save_expense(sender, amount, category, vendor, description, date)

            msg.body(f" Got it! You've spent ₹{amount} on {category}.")
        else:
            msg.body("Couldn't find the amount. Try: 'Spent ₹300 on snacks'.")

        # STEP 2: Handling total expense queries
    elif re.search(r"(how much|what.*spend|total.*spend|spent.*total|how much have i spent)", msg_lower):
        print(" Matched as query:", msg_lower)  # Debugging
        categories = ["food", "essentials", "medicines", "sports"]
        found_category = None

        for cat in categories:
            if cat in msg_lower:
                found_category = cat
                break

        if found_category:
            cursor.execute("SELECT SUM(amount) FROM expenses WHERE user = ? AND category = ?", (sender, found_category))
            total = cursor.fetchone()[0] or 0
            msg.body(f"You’ve spent ₹{total} on {found_category}.")
        else:
            cursor.execute("SELECT SUM(amount) FROM expenses WHERE user = ?", (sender,))
            total = cursor.fetchone()[0] or 0
            msg.body(f"You’ve spent ₹{total} in total.")


    # STEP 3: Help / unknown message
    else:
        msg.body("Try: 'Spent ₹300 on coffee' or 'How much on food?'")

    return str(response)



In [15]:
# STEP 5: Running Flask server in background
import threading

def run_flask():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.61:5000
Press CTRL+C to quit


In [16]:

# STEP 6: Starts ngrok and we will get public URL
from pyngrok import ngrok

ngrok.kill()  # kill old sessions if any
public_url = ngrok.connect(5000)
print(f"🌐 Public webhook URL: {public_url}/webhook")


🌐 Public webhook URL: NgrokTunnel: "https://1c3e-2406-7400-75-99e6-7c99-1715-3d06-97e0.ngrok-free.app" -> "http://localhost:5000"/webhook
